In [71]:
import csv
from pathlib import Path
from dotenv import load_dotenv
import os
import pymongo
import uuid
import random

In [72]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
collections = db_client['db_ai_travel_planner']
tb_city = collections['tb_city']

In [73]:
def load_csv(filepath):
    data = []
    with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:  # Handle encoding!
        reader = csv.reader(csvfile)  # Or csv.DictReader for dictionaries
        header = next(reader)  # Read the header row (if it exists)
        for row in reader:
            data.append(row)  # Or data.append(dict(zip(header, row))) for DictReader
    return header, data  # Return header and data

In [87]:
continent_map = {}  #key: country, value: continent
continents = {} #key: continent, value: 1
#read continent info
header, data = load_csv(Path("./countries.csv"))
for row in data:
    continent = row[12].replace('Americas', 'America').replace('Oceania', 'Australia').lower()
    if continent != '' and continent != 'Polar':
        continent_map[row[1]] = continent
#print(continent_map)

In [ ]:
header, data = load_csv(Path("./city_country_shortened.csv"))
for row in data:
    if row[1] in continent_map:
        #print(row[0] + ',' + row[1] + ',' + continent_map[row[1]])
        a = 1
    else:
        print('Not found continent for this country ------ ' + row[1]) #replace country name in file countries.csv -> run code again to update the map


In [89]:
#export ALL cities that had data in db -> the file show limit to 2 MB
city_continent_map = {} #key: city, value: continent
continents = ['asia', 'america', 'africa', 'australia', 'europe']
for continent in continents:
    num_city_in_continent = 0
    db_city = tb_city.find({'continent':continent, 'error': None})
    for document in db_city:
        if document['name'] not in city_continent_map:
            city_continent_map[document['name']] = document['continent']
            num_city_in_continent += 1
    print(continent + ' : ' + str(num_city_in_continent))

#print(city_continent_map)

asia : 1094
america : 744
africa : 285
australia : 22
europe : 427


"home_cities":{
    "top_banner": "def4a760-7414-4554-8642-a90fa191743d",
    "random_pick":"96981e13-8322-4f46-890b-f68bde008913",
    "europe":["96981e13-8322-4f46-890b-f68bde008913", "def4a760-7414-4554-8642-a90fa191743d", "bff29e88-e132-4bc6-99ce-8b7cc2b8c764"],
      "america":["55818f76-e53d-47eb-9ea5-d1edac65a8e1"
      ],
      "Australia":["fb3a8270-21c7-4b91-a41d-06bf760e6d42"
      ],
      "Africa":["be9ff3d3-2374-4fd6-8d7a-a5b75683cc6e"
      ],
      "Asia":["ca97e354-188a-493f-a21b-345383004e37"
      ]
  }

In [95]:
#select most commented cities for HOMEPAGE
MIN_REVIEW_COUNT = 100
def generate_cities_for_home():
    home_cities = {}
    cities_with_image = []  #list of city UUIDs that has image Urls
    #top banner (randomly pick a city in europe) ~ 100 cities
    query = {'continent':'europe', 'error': None, 'review': {'$gt': MIN_REVIEW_COUNT}}
    top_review_cities = tb_city.find(query).sort({'review': -1}).limit(100)
    total_docs = tb_city.count_documents(query)
    rand_top_index = random.randint(0, total_docs)
    top_banner_city = top_review_cities[rand_top_index]
    cities_with_image.append(top_banner_city['uuid'])
    home_cities['top_banner'] = top_banner_city['uuid']
    print('Top banner city: ' + top_banner_city['name'] + ' country: ' + top_banner_city['country'])
    #random pick (randomly pick a city in america or asia) ~ 100 cities
    query = {'$or': [{'continent':'america'}, {'continent':'asia'}], 'error': None, 'review': {'$gt': MIN_REVIEW_COUNT}}
    random_pick_cities = tb_city.find(query).sort({'review': -1}).limit(100)
    total_docs = tb_city.count_documents(query)
    rand_top_index = random.randint(0, total_docs)
    rand_pick_city = random_pick_cities[rand_top_index]
    cities_with_image.append(rand_pick_city['uuid'])
    home_cities['random_pick'] = rand_pick_city['uuid']
    print('Random pick city: ' + rand_pick_city['name'] + ' country: ' + rand_pick_city['country'])
    #each continent get randomly 20 cities (get 20 cities, sort by review count)
    for continent in continents:
        cities_in_continent = []
        query = {'continent':continent, 'error': None, 'review': {'$gt': MIN_REVIEW_COUNT}}
        pipeline = [
            {"$match": query},
            {"$sample": {"size": 20}}  #X random documents
        ]
        cities = list(tb_city.aggregate(pipeline))
        for city in cities:
            cities_in_continent.append(city['uuid'])
        home_cities[continent] = cities_in_continent
    #
    return home_cities
#test
generate_cities_for_home()

Top banner city: London country: United Kingdom
Random pick city: Mingguang country: China


{'top_banner': '630eeb1c-ddf4-4612-b8a3-6d12e1b39e12',
 'random_pick': '93965f36-efb0-42e6-8b8a-b27ec542eebc',
 'asia': ['bc2356b7-523b-4ef9-be75-4d3e4789cee0',
  'fcab7672-0690-464c-98c5-4cac5a0886f3',
  '0c0cbc9a-c105-4ba8-b0e7-d2fb022fc44b',
  'd61e4573-a88d-4348-abf2-9d46a52e4882',
  'cc0eecf4-dac8-4409-bd1c-8629b1ebbffe',
  '5bf1b512-85af-4048-8770-26f0a3c424e8',
  'c256786a-1b44-4251-b591-3e356785df50',
  'bbd17821-c03f-4fc5-84b4-d6a90322fbd7',
  'd709a283-c327-4c09-98b6-25b9c9b2e088',
  '7569e020-3408-478e-9ed9-5a75a7b14606',
  '1e30f048-7727-4a42-87fd-3bcdf4eb7d0e',
  'ba278bf7-ad5c-43dd-b509-a1927ad3a8d3',
  'c369d9c0-f69a-4935-8868-c9eb85fb1e3f',
  '432a1088-7a5c-4012-97b4-60d3e1ce60dd',
  '35157640-1c21-4513-bcf5-e40a9824db78',
  'c1a0c8d7-bd47-4336-91f1-b0fd5863ea0c',
  '1800a907-49c0-4eaa-8da4-678afd279292',
  'e67a20ad-9564-4ada-9bda-33e526cea4a9',
  '52c54f87-6dd0-4981-880a-34df7355e428',
  '2646d5b2-8ef1-47e8-b04e-d8adc9cc293f'],
 'america': ['bc2b718a-912b-4843-a08c-b3